In [ ]:
%matplotlib notebook
import datetime as datetime
import pandas as pd
import numpy as np
import seaborn 
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import matplotlib.pyplot as plt

#import pandas_datareader.data as web
#import quandl 
#quandl.ApiConfig.api_key = 'DeZUeasSpK9QqNz3UMkk'

In [ ]:
portfolioCSVPath = '/Users/marke/Google Drive/myPapers/myPortfolio.csv'
#portfolioCSVPath = '/Users/marke/Google Drive/myPapers/cathyPortfolio.csv'
cryptoCSVPath = '/Users/marke/Google Drive/myPapers/crypto.csv'

## Funds

In [ ]:
dfPortfolio = pd.read_csv(portfolioCSVPath,header=None)
dfPortfolio

In [ ]:
def df2Value(dfPortfolio):
    ts = TimeSeries(key='7AS7YUCGDVZUSODF', output_format='pandas')
    
    dfDict = {}
    dfDict['CASH'] = (1.0,0,'various cash...')
    
    for desc,symbol, shares in dfPortfolio.values:
        print "getting: " + symbol
        if symbol != 'FDRXX' and symbol !='CORE':
            #dfDict[symbol] = (web.DataReader(symbol, "google", start, end), shares,desc)
            #dfDict[symbol] = quandl.get("WIKI/%s" % symbol, start_date=start, end_date=end)
            
            data, meta_data = ts.get_daily(symbol=symbol,outputsize='compact')
            dfDict[desc] = (data,shares,desc)
            
        else:
            dfDict['CASH'] = (1.0, dfDict['CASH'][1] + shares,'various cash...')
    print "finished..."
    return dfDict 

In [ ]:
result0 = df2Value(dfPortfolio)

In [ ]:
portfolioValueTimeSeries = np.sum(np.array([triple[0]['close'].values * triple[1] 
                                            for key,triple in result0.items() if key !='CASH']),
                                  axis=0) + result0['CASH'][1]

In [ ]:
plt.figure()
plt.plot(portfolioValueTimeSeries)
plt.title("Fund Portfolio Over Previous 100 Days");

In [ ]:
print "stock portfolio: "+ str(portfolioValueTimeSeries[-1])

## Crypto

In [ ]:
dfCrypto = pd.read_csv(cryptoCSVPath,header=None)
dfCrypto

In [ ]:
def dfCrypto2Value(dfCrypto):
    
    cc = CryptoCurrencies(key='7AS7YUCGDVZUSODF', output_format='pandas')
    dfDict = {}
    for desc,symbol, shares in dfCrypto.values:
        print "getting: " + symbol
        #data, meta_data = cc.get_digital_currency_daily(symbol='ETH', market='USD')
        try:
            data, meta_data = cc.get_digital_currency_intraday(symbol=symbol, market='USD')
            dfDict[desc] = (data,shares,desc)
        except:
            "ERROR: " + desc
    print "finished..."
    return dfDict 

In [ ]:
result = dfCrypto2Value(dfCrypto)

In [ ]:
cryptoValueTimeSeries = np.sum(np.array([(triple[0]['. price (USD)'].values[:,0] * triple[1])[-12*48:] for key,triple in result.items()]),axis=0)

In [ ]:
plt.figure()
plt.plot(cryptoValueTimeSeries)
plt.title('Crypto Portfolio over previous 48 hours (increments of 5 minutes)');

In [ ]:
print "crypto portfolio:  " + str(cryptoValueTimeSeries[-1])

In [ ]:
print "total: " + str(cryptoValueTimeSeries[-1] + portfolioValueTimeSeries[-1])